# Generative AI Financial Market Summary using the Gemini Pro Model

## Environment Setup
Install packages.
>Secrets (API keys) are **NOT** used in this demo.

In [ ]:
!pip install --upgrade google-cloud-aiplatform
!pip install yfinance

## Call Model with Tickers and Prompt

Import packages, set variables.

>This demo uses the **Gemini Pro** model which has a much smaller context window limit. Changes to the tickers may trigger an error related to this. The code for using a different model isn't as simple as changing the model name in the variable. So, simply changing it to `text-bison@002` will break the demo.



In [ ]:
# Import necessary libraries
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
import yfinance as yf

# Define the ticker symbols
ticker_symbols = ["DJIA", "NDX"]  # Add the desired stock symbols

# Prompt to send to Vertex AI
disclaimer = "THIS IS NOT FINANCIAL ADVICE. FOR EDUCATIONAL PURPOSES ONLY. Do not take any action based on this information.\n"
summary_request = "Summarize all of this information for an investor audience. At the top should be the opening and closing price. Include the 52-week change fields. The output should be a table."
# Language to use
language = "en-US"
# Model name to use
model_name = "gemini-pro"

# Send prompot to model
def generate(model_name, full_prompt):
    model = GenerativeModel(model_name)
    responses = model.generate_content(
        full_prompt,  # Using the combined prompt
        generation_config={
            "max_output_tokens": 2048,
            "temperature": 0.85,
            "top_p": 1
        },
        stream=True,
    )
    for response in responses:
        print(response.candidates[0].content.parts[0].text)

# Loop through each ticker symbol
for ticker_symbol in ticker_symbols:
    # Fetch data
    stock = yf.Ticker(ticker_symbol)

    # Get basic stock information
    info = stock.info  # Retrieve and store basic stock information

    # Convert all info fields into a string
    info_string = "\n".join([f"{key}: {value}" for key, value in info.items() if value is not None])

    # Get historical market data
    hist = stock.history(period="1mo")  # Change the period as needed
    hist_string = hist.tail().to_string()  # Convert hist to a string

    # Combine everything into a single prompt
    full_prompt = disclaimer + summary_request + "\n" \
                  f"Ticker: {ticker_symbol}\n" \
                  f"Info: {info_string}\n" \
                  f"Recent Stock Data:\n{hist_string}\n"

    # Add a separator for readability when multiple stocks are displayed
    print("\n" + "-"*50 + "\n")

    # Call the generate function with the user-defined model name and the full prompt
    generate(model_name, full_prompt)
